In [129]:
from poolDatasetV2 import PoolDatasetV2
import os
import torch
from torchvision.transforms import v2
from torch.utils.data import DataLoader
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [137]:
model = fasterrcnn_resnet50_fpn(pretrained=False)
num_classes = 2  # 1 class (swimming pool) + background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

c:\Users\romul\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\romul\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [138]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [125]:
transforms = v2.Compose([
    v2.Resize((224, 224)),
    v2.ToImage(),
    #v2.RandomHorizontalFlip(p=1),
    v2.ToDtype(torch.float32, scale=True),
    #v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

ROOT_DIR = os.getcwd() + "\\dataset\\images"

dataset = PoolDatasetV2(ROOT_DIR, transforms)

In [126]:
train, test = dataset.split_Data()
print(len(train), len(test))

4755 2342


In [127]:
len(train[0])

2

In [130]:

def collate_fn(batch):
    images = [item[0] for item in batch]
    boxes = [item[1] for item in batch]

    max_size_labels = max([len(box) for box in boxes])

    for box in boxes:
        if len(box) < max_size_labels:
            toadd = torch.Tensor([0.0, 0.0, 0.0, 0.0])
            box.extend([toadd for i in range(max_size_labels - len(box))])

    return images, boxes


# collate_fn(data)

In [131]:
train_dl = DataLoader(train, batch_size=32, shuffle=True, collate_fn=collate_fn)

test_dl = DataLoader(test, batch_size=1171, shuffle=False, collate_fn=collate_fn)

In [150]:
for i, (images, boxes) in enumerate(train_dl):
    print(i, images, boxes)
    break

0 [tensor([[[0.9020, 0.9020, 0.8745,  ..., 0.6863, 0.6902, 0.6706],
         [0.7765, 0.7882, 0.8000,  ..., 0.6824, 0.6941, 0.7059],
         [0.8471, 0.8471, 0.8745,  ..., 0.6549, 0.6627, 0.6863],
         ...,
         [0.3922, 0.3765, 0.3059,  ..., 0.7412, 0.7176, 0.7020],
         [0.3490, 0.3922, 0.4392,  ..., 0.8000, 0.8000, 0.7294],
         [0.3373, 0.4549, 0.4471,  ..., 0.7647, 0.8039, 0.7882]],

        [[0.6980, 0.6980, 0.6706,  ..., 0.6745, 0.6588, 0.6392],
         [0.5804, 0.5922, 0.6039,  ..., 0.6588, 0.6627, 0.6745],
         [0.6588, 0.6588, 0.6863,  ..., 0.6314, 0.6235, 0.6471],
         ...,
         [0.4157, 0.4000, 0.3294,  ..., 0.7137, 0.6902, 0.6745],
         [0.3765, 0.4196, 0.4667,  ..., 0.7725, 0.7647, 0.6941],
         [0.3686, 0.4863, 0.4784,  ..., 0.7373, 0.7686, 0.7529]],

        [[0.6039, 0.6039, 0.5765,  ..., 0.6157, 0.6078, 0.5882],
         [0.4824, 0.4941, 0.5059,  ..., 0.6039, 0.6118, 0.6235],
         [0.5686, 0.5686, 0.5961,  ..., 0.5765, 0.5765,

# Train

https://medium.com/@natsunoyuki/teaching-a-model-to-become-an-expert-at-locating-cats-and-dogs-in-images-716cdbc8d48f